In [1]:
%%capture
# Install the latest Tensorflow version.
!pip install tensorflow_text
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm
!pip install unidecode
!pip install keras==2.4.3

In [2]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers.merge import multiply, concatenate
import itertools
import datetime
import gensim
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models  import Model,Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, Lambda,Dense,merge
import keras.backend as K
from keras import layers,regularizers
from keras.optimizers import Adadelta, SGD, RMSprop, Adagrad, Adam, Adamax, Nadam
from keras.layers import Input,Embedding, LSTM, Dense, Flatten, Activation, RepeatVector, Permute, Lambda, \
    Bidirectional, TimeDistributed, Dropout, Conv1D, GlobalMaxPool1D
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [3]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/ar_wiki_word2vec-300.rar", outdir="/content/")

     |████████████████████████████████| 77 kB 2.7 MB/s 
patool: Extracting /content/drive/MyDrive/ar_wiki_word2vec-300.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/ar_wiki_word2vec-300.rar
patool:     with cwd='/content/'
patool: ... /content/drive/MyDrive/ar_wiki_word2vec-300.rar extracted to `/content/'.


'/content/'

In [4]:
# File paths
TRAIN_CSV = '/content/ar-en-train.csv'
TEST_CSV = '/content/ar-en.csv'
EMBEDDING_FILE = '/content/drive/MyDrive/RandomShuffle_Skip/randshuffle_5window_skipgram_300size.model'
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
train_df.head()

,Unnamed: 0,english,arabic,sts
0,0,Someone carries a skateboard night on the side...,رجل جالس بمفرده يقرأ على طاولة مستديرة ، خارج ...,0.8
1,1,The women race in the Daytona 500.\n,يتسابق بعض الرجال ضمن مسابقة التزلج,1.0
2,2,Women walk side by side.\n,هناك فتيات يمشين متجاورات,2.6
3,3,The guy with the green shirt jumps up on the g...,يمشي الرجل ذو القميص الأبيض على العشب الطويل م...,2.2
4,4,Two men sitting on the lawn with bananas.\n,ثلاثة رجال يتسكعون عند فرشة بيع الفاكهة,1.4


In [5]:
import nltk
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
ar_stopwords = '''
أنفسنا مثل حيث ذلك بشكل لدى ألا عن إلي ب لنا وقالت فقط الذي الذى ا هذا غير أكثر اي أنا أنت ايضا اذا كيف وكل أو اكثر أي أن منه وكان وفي تلك إن سوف حين نفسها هكذا قبل حول منذ هنا عندما على ضمن لكن فيه عليه قليل صباحا لهم بان يكون بأن أما هناك مع فوق بسبب ما لا هذه و فيها ف ولم ل آخر ثانية انه من الان جدا به بن بعض حاليا بها هم أ كانت هي لها نحن تم أنفسهم ينبغي إلى فان وقد تحت' عند وجود الى فأن الي او قد خارج إنه اى مرة هؤلاء أنها إذا فهي فهى كل يمكن جميع أنفسكم فعل كان ثم لي الآن وقال فى في ديك لم لن له تكون الذين ليس التى التي أنه وان بعد حتى ان دون وأن لماذا يجري كلا إنها لك ضد وإن فهو انها منها أى لديه ولا بين خلال وما اما عليها بعيدا كما نفسي نحو هو نفسك نفسه انت ولن إضافي لقاء وكانت هى فما أيضا إلا معظم ومن إما الا بينما وهي وهو وهى
'''

ar_stopwords=nltk.word_tokenize(ar_stopwords)
print("length of stopwords is: ",len(ar_stopwords))
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
length of stopwords is:  179


In [6]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [27]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
#normalisation
"""
train_df =normalize(train_df, ["sts"])
test_df =normalize(test_df, ["sts"])
"""
stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text
def text_to_arabic_word_list(text):
    search = ["أ", "إ", "آ", "ة", "_", "-", "/", ".", "،", " و ", " يا ", '"', "ـ", "'", "ى", "\\", '\n', '\t',
              '&quot;', '?', '؟', '!']
    replace = ["ا", "ا", "ا", "ه", " ", " ", "", "", "", " و", " يا", "", "", "", "ي", "", ' ', ' ', ' ', ' ? ', ' ؟ ',
               ' ! ']
    text = str(text)

    # remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel, "", text)

    # remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    # trim & split
    text = text.strip()
    text = text.split()
    
    return text
# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec3 = Word2Vec.load('/content/ar_wiki_word2vec')
word2vec = Word2Vec.load(EMBEDDING_FILE)

sent_cols = ['english', 'arabic']
i=0
# Iterate over the sentences only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both sentences of the row
        for sent in sent_cols:
            q2n = []  # q2n -> sentence numbers representation
            if sent =='english':
               sentence=text_to_word_list(row[sent])
            else:
               sentence=text_to_arabic_word_list(row[sent])
            for word in sentence:
              
                # Check for unwanted words
                if word in stops :
                    continue
                if word in ar_stopwords:
                    continue
                if word not in word2vec.wv.vocab :
                   if word not in word2vec3.wv.vocab :
                     i=i+1
                     continue
              
                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace sentences as word to sentences as number representation
            dataset.at[index, sent]= q2n
print(i)           
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.wv.vocab:
        embeddings[index] = word2vec.wv[word]
        
    elif word in word2vec3.wv.vocab:
        embeddings[index] = word2vec3.wv[word]
        
        
del word2vec

128


In [15]:
max_seq_length = max(train_df.english.map(lambda x: len(x)).max(),
                     train_df.arabic.map(lambda x: len(x)).max(),
                     test_df.english.map(lambda x: len(x)).max(),
                     test_df.arabic.map(lambda x: len(x)).max())

# Split to train validation
validation_size = int(len(train_df) * 0.1)
training_size = len(train_df) - validation_size

X = train_df[sent_cols]
Y = train_df['sts']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.english, 'right': X_train.arabic}
X_validation = {'left': X_validation.english, 'right': X_validation.arabic}
X_test = {'left': test_df.english, 'right': test_df.arabic}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [16]:
def cosine_distance(vecs):
    
    y_true, y_pred = vecs
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return K.mean(1 - K.sum((y_true * y_pred), axis=-1))

def cosine_dist_output_shape(shapes):
    shape1, shape2 = shapes
    print((shape1[0], 1))
    return (shape1[0], 1)

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))
    
def exponent_neg_euclidean_distance(left, right):
    """ Helper function for the similarity estimate of the LSTMs outputs"""
    return K.sqrt(K.sum(K.square(left - right), axis=-1, keepdims=True))

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean((1 - y_true) * K.square(y_pred) +  y_true* K.square(K.maximum(margin - y_pred, 0)),axis=-1)

In [24]:
# Model variables

gradient_clipping_norm = 0.1
batch_size =1024
n_epoch =64


# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

# Embedded version of the inputs
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)
# Since this is a siamese network, both sides share the same LSTM
shared_lstm=Sequential()
shared_lstm.add(LSTM(256,return_sequences=True))
shared_lstm.add(Dense(256, activation='relu'))  

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
#malstm_distance = Lambda(function=lambda x: exponent_neg_euclidean_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
malstm_distance= Lambda(cosine_distance, output_shape=cosine_dist_output_shape)([left_output, right_output])

malstm = Model([left_input, right_input], [malstm_distance])

optimizer = Adam(clipnorm=gradient_clipping_norm)
malstm.compile(loss='mse', optimizer=optimizer)

# Start training
training_start_time = time()
malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size,epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))
malstm.save('SiameseLSTM.h5')
print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Epoch 1/64
1/1 [==============================] - 6s 6s/step - loss: 3.7833 - val_loss: 7.0004
Epoch 2/64
1/1 [==============================] - 1s 518ms/step - loss: 5.7139 - val_loss: 6.8589
Epoch 3/64
1/1 [==============================] - 1s 528ms/step - loss: 5.5789 - val_loss: 6.7656
Epoch 4/64
1/1 [==============================] - 1s 562ms/step - loss: 5.4947 - val_loss: 6.6839
Epoch 5/64
1/1 [==============================] - 1s 535ms/step - loss: 5.4318 - val_loss: 6.6233
Epoch 6/64
1/1 [==============================] - 1s 526ms/step - loss: 5.3944 - val_loss: 6.5976
Epoch 7/64
1/1 [==============================] - 1s 528ms/step - loss: 5.3759 - val_loss: 6.5853
Epoch 8/64
1/1 [==============================] - 1s 539ms/step - loss: 5.3683 - val_loss: 6.5755
Epoch 9/64
1/1 [==============================] - 1s 537ms/step - loss: 5.3653 - val_loss: 6.5696
Epoch 10/64
1/1 [==============================] - 1s 518ms/step - loss: 5.3647 - val_loss: 6.5545
Epoch 11/64
1/1 [=====

In [25]:
for dataset, side in itertools.product([X_test], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

sims = malstm.predict([X_test['left'], X_test['right']],1)
print(sims)


[0.21428575 0.5        0.42857146 0.42857146 0.5714286  0.5714286
 0.5        0.28571433 0.21428575 0.14285718 0.21428575 0.28571433
 0.3571429  0.42857146 0.3571429  0.28571433 0.5714286  0.3571429
 0.28571433 0.71428573 0.85714287 0.28571433 0.3571429  0.64285713
 0.42857146 0.28571433 0.14285718 0.42857146 0.42857146 0.28571433
 0.28571433 0.28571433 0.28571433 0.5714286  0.14285718 0.28571433
 0.21428575 0.14285718 0.3571429  0.64285713 0.5        0.5714286
 0.42857146 0.21428575 0.14285718 0.28571433 0.21428575 0.71428573
 1.         0.5        0.21428575 0.21428575 0.28571433 0.3571429
 0.28571433 0.42857146 0.42857146 0.71428573 0.3571429  0.3571429
 0.64285713 0.21428575 0.5        0.21428575 0.5        0.28571433
 0.28571433 0.42857146 0.42857146 0.3571429  0.21428575 0.42857146
 0.28571433 0.3571429  0.28571433 0.28571433 0.28571433 0.21428575
 0.3571429  0.78571427 0.3571429  0.5        0.28571433 0.5714286
 0.3571429  0.21428575 0.28571433 0.3571429  0.21428575 0.5714286
 0

In [26]:
from math import sqrt

import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn.metrics import mean_squared_error

pearson_correlation = scipy.stats.pearsonr(sims,test_df['sts'])[0]
spearman_correlation = scipy.stats.spearmanr(sims,test_df['sts'])[0]


textstr = '\nPearson Correlation=%.1f\nSpearman Correlation=%.1f ' % ( pearson_correlation*100, spearman_correlation*100)
print(textstr)


Pearson Correlation=-16.9
Spearman Correlation=-23.0 
